In [1]:
import numpy as np
import re

In [2]:
f = open("./input", "r")
lines = f.read().strip().splitlines()

In [3]:
grid = np.array([[x for x in l] for l in lines])

In [4]:
from operator import add, sub
def tsub(a,b):
    return tuple(map(sub, a, b))
def tadd(a,b):
    return tuple(map(add, a, b))

In [5]:
from collections import deque

gardens = []
points = deque([(x, y) for y in range(grid.shape[1]) for x in range(grid.shape[0])])


def explore_garden(c):
    garden = [c]
    for np in [tadd(c, x) for x in [(0, 1), (0, -1), (1, 0), (-1, 0)]]:
        # in_bounds = (0 <= np[0] < grid.shape[0] and 0 <= np[1] < grid.shape[1])
        if not np in points or not grid[np] == grid[c]:
            continue
        points.remove(np)
        garden += explore_garden(np)
    return garden


while len(points):
    gardens.append(explore_garden(points.popleft()))

part1 = 0
for garden in gardens:
    fences = 0
    for p in garden:
        for np in [tadd(p, x) for x in [(0, 1), (0, -1), (1, 0), (-1, 0)]]:
            if not np in garden:
                fences += 1
    part1 += fences * len(garden)
part1

1421958

In [6]:
walk = {
    "down": [
        # outside move, inside move, next, count edge?
        ((1, 0), (1, 0), "down", False),
        ((1, 1), (0, 0), "right", True),
        ((0, 0), (1, -1), "left", True),
    ],
    "left": [
        ((0, -1), (0, -1), "left", False),
        ((1, -1), (0, 0), "down", True),
        ((0, 0), (-1, -1), "up", True),
    ],
    "up": [
        ((-1, 0), (-1, 0), "up", False),
        ((-1, -1), (0, 0), "left", True),
        ((0, 0), (-1, 1), "right", True),
    ],
    "right": [
        ((0, 1), (0, 1), "right", False),
        ((-1, 1), (0, 0), "up", True),
        ((0, 0), (1, 1), "down", True),
    ],
}

In [7]:
part1 = []
part2 = []
for garden in gardens:
    visited = deque()
    edges = 0
    fences = 0

    for point in garden:
        start_inner = point
        start_outer = None
        next = None
        directions = [((0, -1), "down"), ((1, 0), "right"), ((-1, 0), "left"), ((0, 1), "up")]
        for np, n in [(tadd(start_inner, x), n) for x, n in directions]:
            if np not in garden and not (*start_inner, *np) in visited:
                start_outer = np
                next = n
                break
        else:
            continue

        inner = start_inner
        outer = start_outer

        while True:
            for o, i, n, c in walk[next]:
                ni = tadd(inner, i)
                no = tadd(outer, o)
                if ni in garden and not no in garden:
                    fences += 1
                    edges += 1 if c else 0
                    inner, outer, next = ni, no, n
                    visited.append(tuple((*inner, *outer)))
                    break
            else:
                raise ValueError("did not found next", garden)
            if inner == start_inner and outer == start_outer:
                break
    part1.append(fences * len(garden))
    part2.append(edges * len(garden))
print(sum(part1))
print(sum(part2))

1421958
885394
